In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/39.OEMPartsOnline'
else:
    print('fuck')

path_root

Mounted at /content/drive


'./drive/MyDrive/code/03.crawler/39.OEMPartsOnline'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(300)

from gevent.lock import BoundedSemaphore
semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import shutil
directory_out = os.path.join(path_root, './file/2.url')
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

from urllib.parse import urlparse

# = = =  = = =  = = =

print('Data Loading...')
list_series_error, df_input = [], pd.read_excel(os.path.join(path_root, './file/1.page.xlsx'),
                                                header=0,
                                                dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)
df_input['SKU Count'] = df_input['SKU Count'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

# = = =  = = =  = = =

def crawler(crawler_tuple):
    global list_series_error, crawler_remain

    crawler_index, crawler_series = crawler_tuple

    # = = =  = = =  = = =

    try:
        list_url = []
        for crawler_cycle in range(5):

            for page in range(1, crawler_series['Page'] + 1):

                crawler_retry = 0
                while True:
                    crawler_retry += 1

                    try:
                        url_request = f'''{crawler_series['Url']}&page={page}'''

                        resp = requests.get(url_request,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(5, 15))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            list_href = html.xpath('//div[@class="catalog-product marketplace-product row"]/div[@class="product-details-col col-md-10 row"]/div[@class="col-xs-12 col-md-8"]/div[@class="product-partnum-category"]/div[@class="catalog-product-id"]/a/@href') + html.xpath('//div[@class="catalog-product row"]/div[@class="product-details-col col-xs-7"]/div[@class="product-partnum-category"]/div[@class="catalog-product-id"]/a/@href')
                            if list_href:
                                break
                    except requests.exceptions.ProxyError as e:
                        if '456 CN Client Forbidden' in str(e):
                            print('Long live the People\'s Republic of China!')
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = =  = = =  = = =

                url_parse = urlparse(crawler_series['Url'])
                list_href = [f'''{url_parse.scheme}://{url_parse.netloc}{href.split('?')[0].strip()}''' for href in list_href]

                # = = =  = = =  = = =

                list_url.extend(list_href)
                list_url = list(dict.fromkeys(list_url))

                # = = =  = = =  = = =

                if crawler_remain % 3 == 0:
                    clear_output()
                print(f'''[第{crawler_cycle + 1}轮] - [第{page}页] - [请求{crawler_retry}次] - [{len(list_url)}/{crawler_series['SKU Count']}] - [剩余{crawler_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

                # = = =  = = =  = = =

                if len(list_url) == crawler_series['SKU Count']:
                    break

            if len(list_url) == crawler_series['SKU Count']:
                break

        # = = =  = = =  = = =

        if not list_url:
            raise

        # = = =  = = =  = = =

        print('Data Outputting...')
        df_correct = pd.DataFrame({'No': range(1, len(list_url) + 1),
                                'Url': list_url})
        df_correct.to_excel(os.path.join(directory_out, f'''./crawler_{crawler_series['No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
        print('Output Finished !')
        print()
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = =  = = =  = = =

    with semaphore_remain:
        crawler_remain -= 1

# = = =  = = =  = = =

pool.map(crawler, df_input.iterrows())

if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(os.path.join(path_root, './file/url_error.xlsx'), index=False)
    print('- Error Occurred -')
    print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

Data Loading...
Loading Done !

总数量：2

[第1轮] - [第1页] - [请求1次] - [18/1375] - [剩余1条] - [2024/11/29 06:59:19] - 2. https://nissan.oempartsonline.com/search?search_str=Resonator&page=1

[第1轮] - [第1页] - [请求1次] - [18/1183] - [剩余1条] - [2024/11/29 06:59:22] - 1. https://toyota.oempartsonline.com/search?search_str=Resonator&page=1

[第1轮] - [第2页] - [请求1次] - [36/1375] - [剩余1条] - [2024/11/29 06:59:24] - 2. https://nissan.oempartsonline.com/search?search_str=Resonator&page=2

[第1轮] - [第3页] - [请求1次] - [54/1375] - [剩余1条] - [2024/11/29 06:59:25] - 2. https://nissan.oempartsonline.com/search?search_str=Resonator&page=3

[第1轮] - [第2页] - [请求1次] - [36/1183] - [剩余1条] - [2024/11/29 06:59:26] - 1. https://toyota.oempartsonline.com/search?search_str=Resonator&page=2

[第1轮] - [第4页] - [请求1次] - [72/1375] - [剩余1条] - [2024/11/29 06:59:27] - 2. https://nissan.oempartsonline.com/search?search_str=Resonator&page=4

[第1轮] - [第3页] - [请求1次] - [54/1183] - [剩余1条] - [2024/11/29 06:59:27] - 1. https://toyota.oempartsonline.